In [107]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
import pandas as pd
import numpy as np


In [109]:
df =  pd.read_csv('/content/drive/MyDrive/D_L/dataset/diabetes.csv')

In [110]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [111]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [112]:
X= df.iloc[:,:-1]
y=df.iloc[:,-1]

In [113]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [114]:
X = scaler.fit_transform(X)
X.shape

(768, 8)

In [115]:
from sklearn.model_selection import train_test_split
X_train , X_test ,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [116]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Input

In [117]:
model_1 = Sequential([
    Input(shape=(8,)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

model_1.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['loss'])

In [118]:
model_2 = Sequential([
    Input(shape=(8,)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

model_2.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [119]:
#model_2.fit(X_train,y_train,batch_size=32,epochs=10)

In [120]:
#model_2.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

In [121]:
#1 . How to select appropriate optimizer
#2. No, of nodes in layer
#3. How to select nombeer of layers
#4. All in all one model

In [122]:
import keras_tuner as kt

In [123]:
###pip install -U keras-tuner

In [124]:
def build_model(hp):

    model = Sequential([
        Input(shape=(8,)),
        Dense(32,activation='relu'),
        Dense(1,activation='sigmoid')
        ])
    #model.compile(optimizer=hp.choice ('optimizer',['adam','sgd','rmsprop','adadelta']))

    optimizer=hp.Choice ('optimizer',values=['adam','sgd','rmsprop','adadelta'])

    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])


    return model

In [125]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir2',
                        project_name='Ravi_thakur'
                        )

In [126]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 11s


In [127]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [128]:
model_3 = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [129]:
model_3.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [130]:
#tuner.results_summary()

In [131]:
#model_3.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

In [145]:
def bulid_model(hp):

  units = hp.Int('units',min_value = 8,max_value = 128,step=8)
  model=([
      Input(shape=(8,)),
      Dense(units=units,activation='relu'),
      Dense(1,activation='sigmoid')
  ])


  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [146]:
tuner3 = kt.RandomSearch(bulid_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='Ravi_thakur')

Reloading Tuner from mydir/Ravi_thakur/tuner0.json


In [147]:
tuner3.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [135]:
#df2 = pd.read_json('/content/mydir/Ravi_thakur/trial_1/build_config.json')

In [136]:
#df2.head()

In [137]:
#df2.describe()

In [138]:
#df3 = pd.read_json('/content/mydir/Ravi_thakur/trial_2/trial.json')

In [148]:
tuner3.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [149]:
model_4 = tuner3.get_best_models(num_models=1)[0]

AttributeError: 'list' object has no attribute 'compile'